In [1]:
import requests
import pandas as pd
from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import boxscore
from datetime import datetime
from datetime import timedelta
import pytz

In [2]:
stat_categories = ['jerseyNum', 'position', 'name', 'minutes',
                   'points', 'reboundsTotal', 'assists', 'steals', 'blocks']
sort_by = ['points']
asc_list = [False]

In [3]:
def get_daily_player_data(date=None):
    board = scoreboard.ScoreBoard()
    timezone = pytz.timezone('EST')
    now = datetime.now(tz=pytz.timezone('EST'))

    if date == None:
        games = board.games.get_dict()

        game_date = board.score_board_date

        daily_stats = []
        for i, game in enumerate(games):
            game_id = game['gameId']
            away = game['awayTeam']['teamTricode']
            home = game['homeTeam']['teamTricode']
            away_score = str(game['awayTeam']['score'])
            home_score = str(game['homeTeam']['score'])
            period = str(game['period'])
            game_clock = game['gameClock']
            start = datetime.strptime(
                game['gameTimeUTC'], "%Y-%m-%dT%H:%M:%S%z")
            start = start.astimezone(timezone)

            if start < now:
                box = boxscore.BoxScore(game_id)
                away_df = pd.DataFrame(box.away_team_player_stats.get_dict())
                home_df = pd.DataFrame(box.home_team_player_stats.get_dict())
                away_df_stats = away_df.join(pd.DataFrame(
                    (away_df.pop('statistics').values.tolist())))
                away_df_stats['team'] = away
                away_df_stats['opp'] = home
                away_df_stats['score'] = away_score + "-" + home_score
                if game['gameStatusText'] != "Final":
                    away_df_stats['game_clock'] = "Q" + period + " - " + game_clock
                else:
                    away_df_stats['game_clock'] = "Final"
                home_df_stats = home_df.join(pd.DataFrame(
                    (home_df.pop('statistics').values.tolist())))
                home_df_stats['team'] = home
                home_df_stats['opp'] = away
                home_df_stats['score'] = home_score + "-" + away_score
                if game['gameStatusText'] != "Final":
                    home_df_stats['game_clock'] = "Q" + period + " - " + game_clock
                else:
                    home_df_stats['game_clock'] = "Final"
                daily_stats.append(away_df_stats)
                daily_stats.append(home_df_stats)

        daily_stats_df = pd.concat(daily_stats)
        daily_stats_df['minutes'] = daily_stats_df['minutes'].replace('PT','',regex=True).replace('M',':',regex=True).replace('S','',regex=True)

        return daily_stats_df.reset_index()

In [6]:
get_daily_player_data()[['name','status']]

,name,status
0,Mikal Bridges,ACTIVE
1,Jae Crowder,ACTIVE
2,Deandre Ayton,ACTIVE
3,Devin Booker,ACTIVE
4,Chris Paul,ACTIVE
5,JaVale McGee,ACTIVE
6,Cameron Payne,ACTIVE
7,Jalen Smith,ACTIVE
8,Landry Shamet,ACTIVE
9,Ish Wainright,ACTIVE


In [5]:
def get_topshot_data():
    topshot_df = pd.read_csv('data/topshot_all_moments.csv')
    return topshot_df

In [144]:
topshot_df = get_topshot_data()

In [145]:
topshot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Time Stamp (EST)   2484 non-null   object 
 1   Player Name        2484 non-null   object 
 2   Set                2484 non-null   object 
 3   Tier               2484 non-null   object 
 4   Series             2484 non-null   object 
 5   Play               2484 non-null   object 
 6   Date of Moment     2484 non-null   object 
 7   Team               2484 non-null   object 
 8   Circulation Count  2484 non-null   int64  
 9   Owned              2484 non-null   int64  
 10  Unique Owners      2484 non-null   int64  
 11  In Packs           2484 non-null   int64  
 12  Minted             2484 non-null   int64  
 13  Held by TS         2484 non-null   int64  
 14  Collector Score    2484 non-null   int64  
 15  Low Ask            2484 non-null   int64  
 16  4h                 2484 

In [147]:
topshot_df

,Time Stamp (EST),Player Name,Set,Tier,Series,Play,Date of Moment,Team,Circulation Count,Owned,...,7d,Listings,Top Shot Debut,Rookie Premiere,Rookie Mint,Rookie Year,Edition State,Play ID,Set ID,Top Shot Link
0,1/16/2022 3:04,Trae Young,Holo Icon,Legendary,2,3 Pointer,2021-01-21 00:30:00+00:00,Atlanta Hawks,58,58,...,-11.70,10,False,False,False,False,CHALLENGE_REWARD_RELEASED,ff51b06a-a677-49f4-98df-9848851ede84,708a6f60-5c93-406e-854f-50dd6734c0dd,https://nbatopshot.com/listings/p2p/708a6f60-5...
1,1/16/2022 3:04,Anthony Davis,Holo Icon,Legendary,2,Jump Shot,2021-01-24 02:00:00+00:00,Los Angeles Lakers,99,93,...,-4.80,23,False,False,False,False,VISIBLE,2a7e80bd-31fd-4a47-b54e-3569658f49c5,708a6f60-5c93-406e-854f-50dd6734c0dd,https://nbatopshot.com/listings/p2p/708a6f60-5...
2,1/16/2022 3:03,Russell Westbrook,Holo Icon,Legendary,2,Layup,2020-12-27 00:00:00+00:00,Washington Wizards,99,91,...,-6.19,28,False,False,False,False,VISIBLE,6c3c0ff8-9a2c-436c-97af-18eb1f74bd5e,708a6f60-5c93-406e-854f-50dd6734c0dd,https://nbatopshot.com/listings/p2p/708a6f60-5...
3,1/16/2022 3:03,Gordon Hayward,Holo Icon,Legendary,2,Layup,2021-01-25 00:00:00+00:00,Charlotte Hornets,99,92,...,4.74,22,False,False,False,False,VISIBLE,0442d216-482b-4a65-930f-159638f5ac4e,708a6f60-5c93-406e-854f-50dd6734c0dd,https://nbatopshot.com/listings/p2p/708a6f60-5...
4,1/16/2022 3:03,T.J. Warren,Holo Icon,Legendary,2,Layup,2020-12-27 01:00:00+00:00,Indiana Pacers,99,92,...,-4.28,25,False,False,False,False,VISIBLE,4be78669-dc48-4633-bcde-2b75fd9cab15,708a6f60-5c93-406e-854f-50dd6734c0dd,https://nbatopshot.com/listings/p2p/708a6f60-5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,1/16/2022 3:28,Lance Stephenson,Base Set,Common,3,3 Pointer,2022-01-06 00:30:00+00:00,Indiana Pacers,12000,0,...,0.00,0,True,NaN,NaN,NaN,VISIBLE,587afb38-dfe7-4f88-82ff-fa44d2000597,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
2480,1/16/2022 3:28,Darius Garland,Base Set,Common,3,Assist,2021-11-30 01:30:00+00:00,Cleveland Cavaliers,60000,0,...,0.00,0,NaN,NaN,NaN,NaN,VISIBLE,30f10dfd-439e-4731-b0b8-ada442ed2a1b,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
2481,1/16/2022 3:28,Wendell Carter Jr.,Base Set,Common,3,Dunk,2021-12-31 00:00:00+00:00,Orlando Magic,60000,0,...,0.00,0,NaN,NaN,NaN,NaN,VISIBLE,e08dff43-369a-443f-86bc-d773dc001262,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
2482,1/16/2022 3:28,Kevin Durant,Base Set,Common,3,Jump Shot,2021-10-24 20:00:00+00:00,Brooklyn Nets,60000,0,...,0.00,0,NaN,NaN,NaN,NaN,VISIBLE,af8a4c1f-12d3-429d-9e2c-e8d8d7a6c5b9,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...


In [206]:
def get_highest_circ_low_ask(topshot_df):
    max_circ = topshot_df[['Player Name', 'Circulation Count']].groupby(['Player Name']).idxmax()
    idx_list = max_circ['Circulation Count'].to_list()
    high_circ_low_ask_df = topshot_df.loc[idx_list]
    return high_circ_low_ask_df

In [207]:
high_circ_low_ask_df = get_highest_circ_low_ask(topshot_df)

In [208]:
high_circ_low_ask_df

,Time Stamp (EST),Player Name,Set,Tier,Series,Play,Date of Moment,Team,Circulation Count,Owned,...,7d,Listings,Top Shot Debut,Rookie Premiere,Rookie Mint,Rookie Year,Edition State,Play ID,Set ID,Top Shot Link
1986,1/16/2022 3:09,A'ja Wilson,WNBA 2021,Fandom,S21,Layup,2021-06-03 23:00:00+00:00,Las Vegas Aces,23396,22882,...,0.00,692,NaN,NaN,NaN,NaN,CHALLENGE_REWARD_RELEASED,626a89f9-7ca7-4400-8bb7-2c0b3978db61,69ae4a5a-6ddf-4bcd-9f7f-0d53f604ecd3,https://nbatopshot.com/listings/p2p/69ae4a5a-6...
1966,1/16/2022 3:10,Aari McDonald,WNBA: Best of 2021,Common,S21,Layup,2021-06-12 00:00:00+00:00,Atlanta Dream,11250,9805,...,0.00,383,True,NaN,True,True,VISIBLE,319631ef-53e8-4191-8a5d-1199a8fbaaa2,391e23a0-f7b6-469e-b1d0-fc24caaebd4d,https://nbatopshot.com/listings/p2p/391e23a0-f...
2359,1/16/2022 3:30,Aaron Gordon,Base Set,Common,3,Dunk,2021-11-24 03:00:00+00:00,Denver Nuggets,60000,29783,...,0.00,1627,NaN,NaN,NaN,NaN,VISIBLE,33febaa8-66c9-498b-a2aa-89af9b043845,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
2456,1/16/2022 3:29,Aaron Holiday,Base Set,Common,3,Handles,2021-12-26 23:00:00+00:00,Washington Wizards,60000,13673,...,0.00,686,NaN,NaN,NaN,NaN,VISIBLE,03112288-0cd6-4d15-8117-c0b3717ec7d0,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
2445,1/16/2022 3:29,Aaron Nesmith,Base Set,Common,3,Dunk,2021-12-21 00:30:00+00:00,Boston Celtics,60000,13571,...,0.00,730,NaN,NaN,NaN,NaN,VISIBLE,98cb5c7b-f804-4303-95a3-171fd7f97e0a,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,1/16/2022 3:16,Zach LaVine,Base Set,Common,2,Dunk,2021-03-17 00:00:00+00:00,Chicago Bulls,40000,36317,...,-12.50,1065,False,False,False,False,VISIBLE,b3b1abdd-7482-4462-b713-af39c5761c9f,208ae30a-a4fe-42d4-9e51-e6fd1ad2a7a9,https://nbatopshot.com/listings/p2p/208ae30a-a...
2067,1/16/2022 3:08,Zach Randolph,Archive Set,Common,S21,Layup,2014-04-17 00:00:00+00:00,Memphis Grizzlies,20000,17439,...,25.00,1095,True,NaN,NaN,NaN,VISIBLE,e5ae87f0-f4f9-4dad-bad2-37cb0cb7aa64,9e89b552-0236-4ffc-ab6b-8cf7c27d46b4,https://nbatopshot.com/listings/p2p/9e89b552-0...
1642,1/16/2022 3:14,Zeke Nnaji,Base Set,Common,2,Dunk,2021-02-11 02:00:00+00:00,Denver Nuggets,12000,10341,...,55.00,187,True,False,True,True,VISIBLE,99ed1382-c37c-4ee0-915f-139f6b58271a,208ae30a-a4fe-42d4-9e51-e6fd1ad2a7a9,https://nbatopshot.com/listings/p2p/208ae30a-a...
2253,1/16/2022 3:33,Ziaire Williams,Base Set,Common,3,3 Pointer,2021-10-21 00:00:00+00:00,Memphis Grizzlies,4000,2060,...,36.36,76,True,True,True,True,VISIBLE,bf0d1f57-613d-40f8-84c6-68f9fef4662b,e7b9646c-9997-46c8-909f-2a2b67389023,https://nbatopshot.com/listings/p2p/e7b9646c-9...
